In [ ]:
!pip install geemap rasterio scikit-learn geopandas pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 31.1 MB/s eta 0:00:00


In [ ]:
!pip install pyproj

In [ ]:
import geemap
import ee
import geopandas as gpd
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
ee.Authenticate()
ee.Initialize(project='ee-garimayadav')

In [ ]:
import geemap
import pandas as pd
import geopandas as gpd
import ee


file_path = '/content/2023_2024_FieldData_CentralPoints_Corrected_with_Height_Class.xlsx'

central_points_df = pd.read_excel(file_path, sheet_name='Data')

# Separate Vermont and Maine/New Hampshire points
vermont_df = central_points_df[central_points_df['States'] == 'VT']
maine_nh_df = central_points_df[central_points_df['States'].isin(['ME', 'NH'])]

# Convert each DataFrame to a GeoDataFrame in the appropriate UTM zone
vermont_gdf = gpd.GeoDataFrame(
    vermont_df,
    geometry=gpd.points_from_xy(vermont_df['Central X'], vermont_df['Central Y']),
    crs="EPSG:32618"  # UTM Zone 18N for Vermont
)

maine_nh_gdf = gpd.GeoDataFrame(
    maine_nh_df,
    geometry=gpd.points_from_xy(maine_nh_df['Central X'], maine_nh_df['Central Y']),
    crs="EPSG:32619"  # UTM Zone 19N for Maine and New Hampshire
)

# Reproject both to EPSG:4326 (WGS84) for compatibility with geemap and map merging
vermont_gdf = vermont_gdf.to_crs(epsg=4326)
maine_nh_gdf = maine_nh_gdf.to_crs(epsg=4326)

# Combine the GeoDataFrames
combined_gdf = pd.concat([vermont_gdf, maine_nh_gdf])

In [ ]:
states = ee.FeatureCollection("TIGER/2018/States")
vermont = states.filter(ee.Filter.eq("NAME", "Vermont"))
maine = states.filter(ee.Filter.eq("NAME", "Maine"))
new_hampshire = states.filter(ee.Filter.eq("NAME", "New Hampshire"))

# Combine the regions into one FeatureCollection
combined_region = vermont.merge(maine).merge(new_hampshire)

In [ ]:
# Function to calculate vegetation indices
def calculate_indices(image):

### STRUCTURE

    # NDVI: Normalized Difference Vegetation Index
    NDVI = image.normalizedDifference(['B8', 'B4']).rename('NDVI')

    NDI45 = image.normalizedDifference(['B4', 'B5']).rename('NDI45')

    #NDI56 = image.normalizedDifference(['B5', 'B6']).rename('NDI56')

    # EVI: Enhanced Vegetation Index
    EVI = image.expression(
        '2.5 * ((NIR - RED) / (1 + NIR + 6 * RED - 7.5 * BLUE ))',
        {'NIR': image.select('B8'), 'RED': image.select('B4'), 'BLUE': image.select('B2')}
    ).rename('EVI')

    # EVI7: Enhanced Vegetation Index 7
    EVI7 = image.expression(
        '2.5 * ((RE3 - RED) / (1 + RE3 + 6 * RED - 7.5 * BLUE ))',
        {'RE3': image.select('B7'), 'RED': image.select('B4'), 'BLUE': image.select('B2')}
    ).rename('EVI7')

    EVI8 = image.expression(
        '2.5 * ((RE4- RED) / (1 + RE4 + 6 * RED - 7.5 * BLUE ))',
        {'RE4': image.select('B8A'), 'RED': image.select('B4'), 'BLUE': image.select('B2')}
    ).rename('EVI8')

    # SR: Simple Ratio (SR)
    MSR = image.expression(
        '((RE3 / RED)-1) / sqrt((RE3 / RED)+1)',
        {'RE3': image.select('B7'), 'RED': image.select('B4'), 'BLUE': image.select('B2')}
    ).rename('MSR')


### Physiology/stress

    # NDII: Normalized Difference Infrared Index
    NDII11 = image.normalizedDifference(['B8A', 'B11']).rename('NDII11')

    NDII12 = image.normalizedDifference(['B8A', 'B12']).rename('NDII12')

    # CRI1: Carotenoid Reflectance Index1 (CRI1)
    CRI1 = image.expression(
        '( 1 / BLUE ) - ( 1 / GREEN )',
        {'BLUE': image.select('B2'), 'GREEN': image.select('B3')}
    ).rename('CRI1')

    #Plant Senescence Reflectance Index (PSRI)
    PSRI = image.expression(
        '(RED - GREEN)/ NIR',
        {'NIR': image.select('B8A'), 'RED': image.select('B4'), 'GREEN': image.select('B3')}
    ).rename('PSRI')

### Biochemistry

    # IRECI: Infrared Red Edge Chlorophyll Index
    IRECI = image.expression(
        '(B7-B4)*(B6/B5)',
        {'B7': image.select('B7'), 'B4': image.select('B4'), 'B6': image.select('B6'),'B5': image.select('B5')}
        #'(NIR - RED) / (RE1 / RE2)',
        #{'NIR': image.select('B8'), 'RE2': image.select('B6'), 'RE1': image.select('B5'),'RED': image.select('B4')}
    ).rename('IRECI')

    # S2REP: Sentinel-2 Red-Edge Photochemical Reflectance Index
    S2REP = image.expression(
        '705 + 35*((((RE3 + RED)/2)-RE1)/(RE2-RE1))',
        {'RE3': image.select('B7'), 'RE2': image.select('B6'), 'RE1': image.select('B5'),'RED': image.select('B4')}
    ).rename('S2REP')


    # GCI: Green Chlorophyll Index
    GCI = image.expression(
        '(RE4 / GREEN) - 1',
        {'RE4': image.select('B8A'), 'GREEN': image.select('B3')}
    ).rename('GCI')

    # GCI: Green Atmospherically Resistant Index (GARI)
    GARI = image.expression(
        '(RE4-GREEN-(BLUE-RED))/(RE4 + GREEN-(BLUE-RED))',
        {'RE4': image.select('B8A'), 'GREEN': image.select('B3'),'RED': image.select('B4'), 'BLUE': image.select('B2')}
    ).rename('GARI')

    GNDVI = image.normalizedDifference(['B8A', 'B3']).rename('GNDVI')

    # NDVIRE: Normalized Difference Vegetation Index Red Edge
    NDVIRE = image.normalizedDifference(['B8A', 'B6']).rename('NDVIRE')


    # TCARI: Transformed Chlorophyll Absorption Ratio Index
    TCARI = image.expression(
        '3*((RE1-RED)-0.2*(RE1-GREEN)*(RE1/RED))',
        {'RE1': image.select('B5'), 'GREEN': image.select('B3'),'RED': image.select('B4'), 'BLUE': image.select('B2')}
    ).rename('TCARI')


    MCARI =  image.expression(
        '(RE2 - RE1) - 0.2 * (RE2 - RED) * RE2 / RE1',
        {'RE1': image.select('B5'), 'RE2': image.select('B6'),'RED': image.select('B4')}
    ).rename('MCARI')



    # Combine all indices into a single image
    indices_image = image.addBands([NDVI, NDI45, EVI, EVI7, EVI8, MSR, CRI1,NDII11,NDII12, PSRI, GCI, GARI, GNDVI, IRECI, S2REP, NDVIRE, TCARI, MCARI])


    return indices_image


In [ ]:
# Define the date ranges for each month
date_ranges = {
    # "Apr": ("2024-04-01", "2024-04-30"),
    # "May": ("2024-04-25", "2024-06-05"),
    # "Jun": ("2024-05-25", "2024-07-08"),
    # "Jul": ("2024-06-25", "2024-07-31"),
    # "Aug": ("2024-07-27", "2024-09-04"),
    # "Sep": ("2024-09-01", "2024-09-30"),
    # "Oct": ("2024-09-24", "2024-11-05"),
    # 'Nov': ('2024-10-25', '2024-12-05'),


    "Apr": ("2023-03-25", "2023-05-05"),
    "May": ("2023-05-01", "2023-05-31"),
    "Jun": ("2023-05-25", "2023-07-05"),
    "Jul": ("2023-06-25", "2023-08-05"),
    "Aug": ("2023-08-01", "2023-08-31"),
    "Sep": ("2023-09-01", "2023-09-30"),
    "Oct": ("2023-10-01", "2023-10-31"),
    'Nov': ('2023-10-25', '2023-12-05'),


    # "Apr_May": ("2024-04-01", "2024-05-31"),
    # "Jun_Jul": ("2024-06-01", "2024-07-31"),
    # "Sept": ("2024-09-01", "2024-09-30"),
    # "Oct": ("2024-10-01", "2024-10-31"),
    # "Nov_Dec": ("2024-11-01", "2024-12-31"),
}


# Function to process Sentinel-2 data for a given date range
# def process_month(date_range):
#     collection = (
#         ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
#         .filterDate(*date_range)
#         .filterBounds(combined_region)
#         .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 40))
#         .map(mask_s2_sr_clouds)
#         .map(calculate_indices)
#     )
#     return collection.reduce(ee.Reducer.mean()).clip(combined_region)

  # or .percentile(25)

def process_month(date_range):
    collection = (
        ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
        .filterDate(*date_range)
        .filterBounds(combined_region)
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 40))
        .map(mask_s2_sr_clouds)  # Ensure cloud masking works well
        .map(calculate_indices)  # Compute NDVI and other indices before aggregation
    )

    # Take the mean of the images in the collection
    mean_image = collection.median().clip(combined_region)

    return mean_image


def mask_s2_sr_clouds(image):
    """
    Masks clouds and shadows using both SCL and band thresholds.
    """
    # Scene Classification Layer (SCL) for cloud and shadow masking
    scl = image.select('SCL')

    # Mask out clouds, shadows, and cirrus using SCL
    cloud_shadow_mask = scl.neq(3)  # 3 = cloud shadow
    cloud_mask = scl.neq(8).And(scl.neq(9))  # 8 = cloud medium, 9 = cloud high
    cirrus_mask = scl.neq(10)  # 10 = cirrus
    scl_mask = cloud_shadow_mask.And(cloud_mask).And(cirrus_mask)

    # Band-based masking (e.g., to remove bright or dark pixels)
    cloud_b4 = image.select('B4').lt(1600)  # Mask pixels where B4 > 1600 (bright)
    cloud_b5 = image.select('B5').gt(500)  # Mask pixels where B5 < 500 (dark)
    cloud_b8 = image.select('B8').gt(1200)  # Mask pixels where B8 < 1200 (dark)
    band_mask = cloud_b4.And(cloud_b5).And(cloud_b8)

    # Combine SCL and band masks
    combined_mask = scl_mask.And(band_mask)

    # Apply the combined mask and scale reflectance values
    return image.updateMask(combined_mask).divide(10000)

# Process each month's data and store in a dictionary
monthly_composites = {month: process_month(date_range) for month, date_range in date_ranges.items()}

# Function to extract indices for a given composite and add as properties
def extract_indices_for_month(feature, composite, prefix):
    reducer = ee.Reducer.first()
    indices = composite.reduceRegion(
        reducer=reducer,
        geometry=feature.geometry(),
        scale=10,
        maxPixels=1e9
    )
    # Get keys from the ee.Dictionary and map them to prefixed properties
    keys = indices.keys()
    prefixed_properties = keys.map(lambda key: ee.String(key).cat("_").cat(prefix))
    values = indices.values(keys)
    # Use combine to set prefixed properties with corresponding values
    return feature.set(ee.Dictionary.fromLists(prefixed_properties, values))

features = []
for _, row in combined_gdf.iterrows():
    feature = ee.Feature(
        ee.Geometry.Point([row.geometry.x, row.geometry.y]),
        {
                'Plot No': int(row['Plot No']),
                'State': row['States'],
                'Date': row['Date'],
                'Location': row['Location'],
                'Central_X': row['Central X'],
                'Central_Y': row['Central Y']

            }
    )
    features.append(feature)
central_points_ee = ee.FeatureCollection(features)

# Add indices from each month to the feature collection
for month, composite in monthly_composites.items():
    central_points_ee = central_points_ee.map(lambda feature: extract_indices_for_month(feature, composite, month))

# Convert the enriched FeatureCollection to a dictionary
points_dict_with_monthly_indices = central_points_ee.getInfo()

# Convert to a DataFrame
data_m = pd.DataFrame([
    {**feature['properties']} for feature in points_dict_with_monthly_indices['features']
])

# ntree mtry grod search
# randomisation

In [ ]:
# Handle Missing Values (if any)
data=[]
#data = data_m.dropna()

# Save the DataFrame to a file or explore it
print("Sample data with monthly indices:\n", data_m)

Sample data with monthly indices:
      AOT_Apr  AOT_Aug  AOT_Jul  AOT_Jun  AOT_May  AOT_Oct  AOT_Sep  B11_Apr  \
0     0.0175   0.0093  0.02345  0.01160  0.00785   0.0145  0.01160   0.1086   
1     0.0175   0.0093  0.02350  0.01165  0.00785   0.0145  0.01160   0.1635   
2     0.0181   0.0070  0.01920  0.01165  0.00785   0.0146  0.01170   0.2188   
3     0.0181   0.0070  0.01920  0.01610  0.00700   0.0146  0.01170   0.2082   
4     0.0181   0.0070  0.01930  0.01600  0.00790   0.0146  0.01170   0.1712   
..       ...      ...      ...      ...      ...      ...      ...      ...   
160   0.0192   0.0084  0.01860  0.01860  0.02580   0.0324  0.02100   0.0891   
161   0.0192   0.0084  0.01650  0.01860  0.02580   0.0324  0.02100   0.0977   
162   0.0192   0.0084  0.01650  0.01860  0.01860   0.0324  0.02175   0.1244   
163   0.0192   0.0084  0.01860  0.01860  0.01860   0.0323  0.02165   0.1318   
164   0.0192   0.0084  0.01860  0.01860  0.01860   0.0324  0.02165   0.1138   

     B11_Aug  B1

In [ ]:
# Export to a single Excel sheet
output_file = 'indices_values_2023.xlsx'
data_m.to_excel(output_file, sheet_name='All_Indices')

print(f"Data exported successfully to {output_file}")


Data exported successfully to indices_values_2023_1.xlsx


In [ ]:
# Load DEM and calculate slope and aspect (do this once outside the function)
dem = ee.Image('USGS/SRTMGL1_003')  # Digital Elevation Model
slope = ee.Terrain.slope(dem).rename('slope')
aspect = ee.Terrain.aspect(dem).rename('aspect')

# Add radar geometry (elevation angle, slope, aspect)
def add_radar_geometry(image):
    elevation_angle = image.select('angle')  # Radar incidence angle
    terrain_slope = slope.multiply(3.14159265359 / 180)  # Convert slope to radians
    terrain_aspect = aspect.multiply(3.14159265359 / 180)  # Convert aspect to radians
    return image.addBands(elevation_angle).addBands(terrain_slope).addBands(terrain_aspect)

# Terrain flattening function
def terrain_flattening(image):
    radar_incidence_angle = image.select('angle').multiply(3.14159265359 / 180)  # Convert to radians
    terrain_slope = image.select('slope')
    cos_theta = radar_incidence_angle.cos()
    cos_alpha = terrain_slope.cos()

    # Radiometric terrain normalization
    vv_normalized = image.select('VV').divide(cos_theta).multiply(cos_alpha).rename('VV_flattened')
    vh_normalized = image.select('VH').divide(cos_theta).multiply(cos_alpha).rename('VH_flattened')

    return image.addBands([vv_normalized, vh_normalized])

# Function to preprocess Sentinel-1 data for a given date range
def preprocess_s1(date_range):
    # Filter Sentinel-1 collection by date and region
    s1_collection = (
        ee.ImageCollection('COPERNICUS/S1_GRD')
        .filterDate(*date_range)
        .filter(ee.Filter.eq('instrumentMode', 'IW'))  # Interferometric Wide Swath
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
        .filterBounds(combined_region)
    )

    # Check if the collection is empty
    if s1_collection.size().getInfo() == 0:
        print(s1_collection.size().getInfo())
        print(f"No Sentinel-1 images found for date range: {date_range}")
        return None

    # Add radar geometry and terrain flattening
    s1_preprocessed = s1_collection.map(add_radar_geometry).map(terrain_flattening)

    # Create a median composite
    s1_composite = s1_preprocessed.median().clip(combined_region)

    # Calculate derived metrics
    vv_vh_ratio = s1_composite.expression(
        'VV / VH', {
            'VV': s1_composite.select('VV_flattened'),
            'VH': s1_composite.select('VH_flattened')
        }
    ).rename('VV_VH_Ratio')

    sqrt_vv_vh = s1_composite.expression(
        'sqrt(VV / (VH + 1e-6))', {
            'VV': s1_composite.select('VV_flattened'),
            'VH': s1_composite.select('VH_flattened')
        }
    ).rename('Sqrt_VV_VH')

    sqrt_vv_times_vh = s1_composite.expression(
        'sqrt(VV * VH)', {
            'VV': s1_composite.select('VV_flattened'),
            'VH': s1_composite.select('VH_flattened')
        }
    ).rename('Sqrt_VV_Times_VH')

    vv_times_vh = s1_composite.expression(
        'VV * VH', {
            'VV': s1_composite.select('VV_flattened'),
            'VH': s1_composite.select('VH_flattened')
        }
    ).rename('VV_Times_VH')

    # Combine bands with derived metrics
    combined_s1_image = (
        s1_composite
        .addBands(vv_vh_ratio)
        .addBands(sqrt_vv_vh)
        .addBands(sqrt_vv_times_vh)
        .addBands(vv_times_vh)
    )

    return combined_s1_image


# Process each month's Sentinel-1 data and store in a dictionary
monthly_s1_composites = {}
for month, date_range in date_ranges.items():
    print(f"Processing {month}...")
    print(monthly_s1_composites)
    monthly_s1_composites[month] = preprocess_s1(date_range)

Processing Apr...
{}
Processing May...
{'Apr': <ee.image.Image object at 0x7f20f5886950>}
Processing Jun...
{'Apr': <ee.image.Image object at 0x7f20f5886950>, 'May': <ee.image.Image object at 0x7f20f5a3b750>}
Processing Jul...
{'Apr': <ee.image.Image object at 0x7f20f5886950>, 'May': <ee.image.Image object at 0x7f20f5a3b750>, 'Jun': <ee.image.Image object at 0x7f20f5a52e90>}
Processing Aug...
{'Apr': <ee.image.Image object at 0x7f20f5886950>, 'May': <ee.image.Image object at 0x7f20f5a3b750>, 'Jun': <ee.image.Image object at 0x7f20f5a52e90>, 'Jul': <ee.image.Image object at 0x7f20f3073dd0>}
Processing Sep...
{'Apr': <ee.image.Image object at 0x7f20f5886950>, 'May': <ee.image.Image object at 0x7f20f5a3b750>, 'Jun': <ee.image.Image object at 0x7f20f5a52e90>, 'Jul': <ee.image.Image object at 0x7f20f3073dd0>, 'Aug': <ee.image.Image object at 0x7f20f3072150>}
Processing Oct...
{'Apr': <ee.image.Image object at 0x7f20f5886950>, 'May': <ee.image.Image object at 0x7f20f5a3b750>, 'Jun': <ee.imag

In [ ]:
def extract_s1_indices_for_month(feature, composite, prefix):
    reducer = ee.Reducer.first()
    indices = composite.reduceRegion(
        reducer=reducer,
        geometry=feature.geometry(),
        scale=10,
        maxPixels=1e9
    )
    # Get keys from the ee.Dictionary and map them to prefixed properties
    keys = indices.keys()
    prefixed_properties = keys.map(lambda key: ee.String(key).cat("_").cat(prefix))
    values = indices.values(keys)

    # Use combine to set prefixed properties with corresponding values
    return feature.set(ee.Dictionary.fromLists(prefixed_properties, values))

In [ ]:


# Add indices from each month to the feature collection
for month, composite in monthly_s1_composites.items():
    central_points_ee = central_points_ee.map(lambda feature: extract_s1_indices_for_month(feature, composite, month))
    #print(central_points_ee.getInfo())


# Convert Sentinel-1 FeatureCollection to a dictionary
s1_points_dict = central_points_ee.getInfo()

#print(s1_points_dict)

# Convert Sentinel-1 data to Pandas DataFrame
s1_data = pd.DataFrame([
    {**feature['properties']} for feature in s1_points_dict['features']
])


# Handle missing values for Sentinel-1 data
#s1_data = s1_data.dropna()

In [ ]:
print(s1_points_dict)

{'type': 'FeatureCollection', 'columns': {}, 'features': [{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-73.0013162952975, 44.446043469940015]}, 'id': '0', 'properties': {'AOT_Apr': 0.017500000074505806, 'AOT_Aug': 0.00930000003427267, 'AOT_Jul': 0.02345000021159649, 'AOT_Jun': 0.011599999852478504, 'AOT_May': 0.007849999703466892, 'AOT_Oct': 0.014499999582767487, 'AOT_Sep': 0.011599999852478504, 'B11_Apr': 0.10859999805688858, 'B11_Aug': 0.11819999665021896, 'B11_Jul': 0.1408499926328659, 'B11_Jun': 0.13244999945163727, 'B11_May': 0.1421000063419342, 'B11_Oct': 0.11640000343322754, 'B11_Sep': 0.10939999669790268, 'B12_Apr': 0.061000000685453415, 'B12_Aug': 0.04809999838471413, 'B12_Jul': 0.06024999916553497, 'B12_Jun': 0.061650000512599945, 'B12_May': 0.06584999710321426, 'B12_Oct': 0.04969999939203262, 'B12_Sep': 0.04349999874830246, 'B1_Apr': 0.0007999999797903001, 'B1_Aug': 0.02979999966919422, 'B1_Jul': 0.04190000146627426, 'B1_Jun': 0.04149999842047691, 'B1_Ma

In [ ]:


# Export the combined data to an Excel file
with pd.ExcelWriter('2024_indices_values.xlsx', engine='openpyxl', mode='w') as writer:
    s1_data.to_excel(writer, sheet_name='All_Indices', index=False)

print(f"Data from Sentinel-1 and Sentinel-2 exported successfully to 'indices_values_2024.xlsx'")

Data from Sentinel-1 and Sentinel-2 exported successfully to 'indices_values_2024.xlsx'


In [ ]:
# Load USGS SRTM DEM
dem = ee.Image('USGS/SRTMGL1_003')

# Resample DEM to 10m resolution
dem_resampled = dem.resample('bilinear').reproject(crs=dem.projection(), scale=10)

# Calculate slope
slope_10m = ee.Terrain.slope(dem_resampled).rename('slope_10m')

# Combine elevation and slope into a single image
terrain_data = dem_resampled.rename('elevation_10m').addBands(slope_10m)


In [ ]:
def extract_terrain_data(feature):
    reducer = ee.Reducer.first()
    terrain_values = terrain_data.reduceRegion(
        reducer=reducer,
        geometry=feature.geometry(),
        scale=10,
        maxPixels=1e9
    )
    return feature.set(terrain_values)

# Apply terrain data extraction function to features
terrain_points_ee = ee.FeatureCollection(features).map(extract_terrain_data)

# Convert FeatureCollection to a dictionary
terrain_points_dict = terrain_points_ee.getInfo()

# Convert to Pandas DataFrame
terrain_data_df = pd.DataFrame([
    {**feature['properties']} for feature in terrain_points_dict['features']
])

# Handle missing values
#terrain_data_df = terrain_data_df.dropna()


# # Debug: Check terrain DataFrame
# print("Columns in Terrain DataFrame:", terrain_data_df.columns)
# print("Sample Terrain Data:\n", terrain_data_df.head())


In [ ]:
# Merge terrain data with existing Sentinel data

final_data = pd.merge(s1_data, terrain_data_df, on='Plot No')
final_data_cleaned = final_data.dropna(axis=1, how='any')
# Export final data to Excel
output_file = '23_data.xlsx'
with pd.ExcelWriter(output_file, engine='openpyxl', mode='w') as writer:
    final_data.to_excel(writer, sheet_name='All_Data', index=False)

print(f"Final data with elevation and slope exported successfully to {output_file}")

Final data with elevation and slope exported successfully to 23_data.xlsx


In [ ]:
print(final_data)

     AOT_Apr  AOT_Aug  AOT_Jul  AOT_Jun  AOT_May  AOT_Oct  AOT_Sep  B11_Apr  \
0    0.01750   0.0093  0.02345  0.01160  0.00785  0.01450   0.0116  0.10860   
1    0.01750   0.0093  0.02350  0.01165  0.00785  0.01450   0.0116  0.16350   
2    0.01810   0.0070  0.01920  0.01165  0.00785  0.01460   0.0117  0.21880   
3    0.01810   0.0070  0.01920  0.01610  0.00700  0.01460   0.0117  0.20820   
4    0.01810   0.0070  0.01930  0.01600  0.00790  0.01460   0.0117  0.17120   
..       ...      ...      ...      ...      ...      ...      ...      ...   
221  0.01255   0.0094  0.01600  0.00830  0.02245  0.01145   0.0201  0.09060   
222  0.01255   0.0094  0.01600  0.00830  0.02240  0.01120   0.0201  0.09040   
223  0.01260   0.0100  0.01830  0.00830  0.02240  0.01135   0.0201  0.11160   
224  0.01265   0.0100  0.01830  0.00830  0.02230  0.01135   0.0201  0.10610   
225  0.01260   0.0100  0.01830  0.00820  0.01360  0.01130   0.0201  0.09935   

     B11_Aug  B11_Jul  ...  MSK_CLASSI_CIRRUS_Nov  